# Guidance ReAct Agent

This notebook mirrors the Outlines ReAct agent example using the [guidance](https://github.com/guidance-ai/guidance) library. We keep the same pair of tools (Wikipedia search and a calculator), the same JSON decision contract, and the same outer control loop so we can compare behaviors side-by-side.


> Requirements: `pip install guidance httpx pydantic` (plus whichever model backend you want to use). If you rely on a hosted API such as OpenAI, remember to export the appropriate API key before running the cells below.


In [ ]:
import datetime as dt
import json
from ast import literal_eval
from enum import Enum

import guidance
import httpx
from guidance import assistant, models, system, user
from guidance import json as gen_json
from guidance.library import capture
from guidance.models import Model
from pydantic import BaseModel, ConfigDict, Field

In [2]:
# Select a guidance model backend.
# Uncomment the option that fits your environment, or replace with your own setup.

# Example: OpenAI API (set OPENAI_API_KEY beforehand).
MODEL = models.OpenAI("gpt-4o-mini")

# Example: local Transformers model.
# from transformers import AutoModelForCausalLM, AutoTokenizer
# HF_MODEL = AutoModelForCausalLM.from_pretrained("microsoft/Phi-4-mini-instruct")
# HF_TOKENIZER = AutoTokenizer.from_pretrained("microsoft/Phi-4-mini-instruct")
# MODEL = models.Transformers(HF_MODEL, HF_TOKENIZER)

# Example: llama.cpp model.
# from llama_cpp import Llama
# LLAMA = Llama("/path/to/model.gguf", n_ctx=8192)
# MODEL = models.LlamaCpp(LLAMA)

In [ ]:
# Tool implementations (identical to the Outlines tutorial with extra guards)


def wikipedia(query: str) -> str:
    """Return the first Wikipedia search snippet for the provided query."""
    if not query.strip():
        return "Invalid query: provide a non-empty search term."
    response = httpx.get(
        "https://en.wikipedia.org/w/api.php",
        params={
            "action": "query",
            "list": "search",
            "srsearch": query,
            "format": "json",
        },
        timeout=10,
    )
    response.raise_for_status()
    data = response.json()
    search_results = data.get("query", {}).get("search", [])
    if not search_results:
        return "No results."
    return search_results[0].get("snippet", "No snippet available.")


def calculate(expression: str) -> str:
    """Evaluate a basic arithmetic expression using literal evaluation safeguards."""
    if not expression.strip():
        return "Invalid expression: provide a non-empty arithmetic expression."
    try:
        result = literal_eval(expression)
    except (ValueError, SyntaxError, TypeError) as exc:
        return f"Error evaluating expression: {exc}"
    return str(result)


TOOL_REGISTRY = {
    "wikipedia": wikipedia,
    "calculate": calculate,
}

In [ ]:
class Action(str, Enum):
    """Enumerate the available tools exposed to the agent."""

    wikipedia = "wikipedia"
    calculate = "calculate"


class ReasonAndAct(BaseModel):
    """Intermediate decision payload instructing the agent to run a tool."""

    model_config = ConfigDict(extra="forbid")
    Scratchpad: str = Field(..., description="Notes derived from prior observations")
    Thought: str = Field(..., description="Your reasoning about the next step")
    Action: Action
    Action_Input: str = Field(..., description="Arguments for the selected action")


class FinalAnswer(BaseModel):
    """Terminal payload containing the final answer and supporting scratchpad."""

    model_config = ConfigDict(extra="forbid")
    Scratchpad: str = Field(..., description="Notes that justify the final answer")
    Final_Answer: str = Field(
        ..., description="Grounded answer to the original question"
    )


class Decision(BaseModel):
    """Discriminated union between an action request and a final answer."""

    model_config = ConfigDict(extra="forbid")
    Decision: ReasonAndAct | FinalAnswer

In [5]:
# Sanity check: ensure schema is JSON-schema compliant for OpenAI
print(json.dumps(Decision.model_json_schema(), indent=2))

{
  "$defs": {
    "Action": {
      "enum": [
        "wikipedia",
        "calculate"
      ],
      "title": "Action",
      "type": "string"
    },
    "FinalAnswer": {
      "additionalProperties": false,
      "properties": {
        "Scratchpad": {
          "description": "Notes that justify the final answer",
          "title": "Scratchpad",
          "type": "string"
        },
        "Final_Answer": {
          "description": "Grounded answer to the original question",
          "title": "Final Answer",
          "type": "string"
        }
      },
      "required": [
        "Scratchpad",
        "Final_Answer"
      ],
      "title": "FinalAnswer",
      "type": "object"
    },
    "ReasonAndAct": {
      "additionalProperties": false,
      "properties": {
        "Scratchpad": {
          "description": "Notes derived from prior observations",
          "title": "Scratchpad",
          "type": "string"
        },
        "Thought": {
          "description": "Your reaso

In [ ]:
SCHEMA_JSON = json.dumps(Decision.model_json_schema(), indent=2)

SYSTEM_PROMPT = (
    "You are a ReAct agent with access to two tools:\n"
    "1. wikipedia(query): search Wikipedia and return the top result snippet.\n"
    "2. calculate(expression): evaluate a Python arithmetic expression.\n\n"
    "Respond **only** with JSON that matches this schema:\n"
    f"{SCHEMA_JSON}\n\n"
    "Guidelines:\n"
    "- Track useful facts from observations in the Scratchpad.\n"
    "- When you need external information, pick an Action and supply Action_Input.\n"
    "- After an Observation is provided,\n"
    "  fold it into the next Scratchpad before deciding again.\n"
    "- Finish with Final_Answer once you can answer the user's question.\n"
    "- Do not invent tools and do not output anything outside the JSON envelope."
)

In [ ]:
@guidance(stateless=True)
def generate_decision(
    lm: Model,
    *,
    name: str = "decision_payload",
    schema: type[Decision] = Decision,
) -> Model:
    """Capture a structured decision from the language model using the schema."""
    return lm + capture(gen_json(schema=schema), name=name)


def run_tool(action: str, action_input: str) -> str:
    """Execute a registered tool and surface any execution errors."""
    func = TOOL_REGISTRY.get(action)
    if func is None:
        return f"Unknown tool: {action}"
    try:
        return str(func(action_input))
    except (ValueError, RuntimeError, httpx.HTTPError, TypeError) as exc:
        return f"Tool error: {exc}"

In [ ]:
def react_agent(question: str, *, max_turns: int = 5) -> str:
    """Run the ReAct loop with guidance using the shared JSON contract."""
    lm = MODEL
    previous_actions: set[str] = set()
    repeat_message = (
        "You already ran that action. Choose a different input or produce a "
        "Final_Answer if you are ready."
    )

    with system():
        lm += SYSTEM_PROMPT

    with user():
        today = dt.datetime.now(dt.UTC).date().isoformat()
        lm += f"Today is {today}. Question: {question}"

    for turn in range(max_turns):
        decision_name = f"decision_{turn}"

        with assistant():
            lm += generate_decision(name=decision_name)

        decision_payload = json.loads(lm[decision_name])["Decision"]
        scratchpad = decision_payload.get("Scratchpad", "")

        if "Final_Answer" in decision_payload:
            final_answer = decision_payload["Final_Answer"]
            print(f"Scratchpad: {scratchpad}")
            print(f"Final Answer: {final_answer}")
            return final_answer

        action = decision_payload["Action"]
        action_input = decision_payload["Action_Input"]
        signature = f"{action}:{action_input}"

        print(f"Scratchpad: {scratchpad}")
        print(f" -- running {action}: {action_input}")

        if signature in previous_actions:
            observation = repeat_message
        else:
            observation = run_tool(action, action_input)
            previous_actions.add(signature)

        if observation.startswith(("Invalid", "Error", "Tool error")):
            observation += (
                " Provide a valid input or produce a Final_Answer if you no "
                "longer need the tools."
            )

        summary_hint = (
            "If you have enough information to answer the question, output a "
            "Final_Answer summarizing the borders of England."
        )
        with user():
            lm += f"Observation: {observation}\n{summary_hint}"

        print(f"Observation: {observation}")

    fallback_message = (
        "I am sorry, but I am unable to answer your question. Please provide "
        "more information or a different question."
    )
    print(f"Final Answer: {fallback_message}")
    return "No answer found"

In [9]:
react_agent("What's 2 to the power of 10?")

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

Scratchpad: I need to calculate 2 to the power of 10 to answer the question.
 -- running calculate: 2**10
Observation: You already ran that action. TRY A DIFFERENT ACTION INPUT or produce a Final_Answer if you are ready.
Scratchpad: I have previously calculated 2 to the power of 10, which is 1024.
 -- running calculate: 2**10
Observation: You already ran that action. TRY A DIFFERENT ACTION INPUT or produce a Final_Answer if you are ready.
Scratchpad: The question relates to answering 'What's 2 to the power of 10?' which equals 1024. I must now pivot to address the new request regarding the borders of England, as the previous task does not need to be repeated.
 -- running wikipedia: Borders of England
Observation: You already ran that action. TRY A DIFFERENT ACTION INPUT or produce a Final_Answer if you are ready.
Scratchpad: I need to summarize the borders of England, focusing on its geographical boundaries without running previous actions again.
 -- running wikipedia: England
Observat

'No answer found'

In [10]:
react_agent("What does England share borders with?")

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

Scratchpad: 
 -- running wikipedia: England borders
Observation: You already ran that action. TRY A DIFFERENT ACTION INPUT or produce a Final_Answer if you are ready.
Scratchpad: England is part of the United Kingdom and shares its land border only with Scotland to the north and Wales to the west. It is also surrounded by water on other sides, including the North Sea, the English Channel, and the Celtic Sea.
Final Answer: England shares its land borders with Scotland to the north and Wales to the west. It is surrounded by water on all other sides.


'England shares its land borders with Scotland to the north and Wales to the west. It is surrounded by water on all other sides.'